This notebook is dedicated to improving, or building a brand new summary section.

In [1]:
with open("job_desc.md") as f:
    description = f.read()
with open("resume.md") as f:
    resume = f.read()
title = 'django developer'

In [2]:
from util import chunk_markdown, cut_sections

In [3]:
sections = cut_sections(resume)

In [4]:
summary = sections['summary']

Two different independent chains will be built to improve summary sections:
1. Take existing summary and tailor it for job description.
2. Use the revised job experience section to extract demonstrations of required skills (from job description) as statements, statistics, or stories. The list of demonstrations will then undergo elimination, condensation, and ordering to build a summary section.

Summary is built by default, in the event that the input document doesn't include a summary.

## Improve existing summary section

This assumes that the writer provided sufficient text in the existing summary and presented strong enough arguments.
 
The role hallucination serves has not been verified. A certain degree of "creativity" is desired to truly reframe the existing content to retain appeal while conveying confidence and competence. However, the algorithm stating facts that aren't true to fit the job description is not desired.

In [5]:

from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import SequentialChain


def improve_summary_chain() -> SequentialChain:
    llm = ChatOpenAI(temperature=.2, model_name="gpt-3.5-turbo")

    appeal_prompt = PromptTemplate.from_template("""
    You will be given a resume section. You will also be given a job description.
    Improve the given resume section to be more appealing to a recruiter looking to fill the given job description.
    The goal is to convey confidence and competence.
    Return only the improved summary, with no header.
    
    Resume Section:
    {section}
    
    Job Description:
    {desc}
    """)
    appeal_chain = LLMChain(prompt=appeal_prompt, llm=llm, output_key="improved")

    summ_prompt = PromptTemplate.from_template("""
    Condense the following resume summary section using {format}.
    The summary should remain appealing to a recruiter and be less than 260 characters.
    
    Resume section:
    {improved}
    """,
                                               partial_variables={'format': 'a mix of bullets and 1 short paragraph'})
    summ_chain = LLMChain(prompt=summ_prompt, llm=llm, output_key="summarized")

    return SequentialChain(
        chains=[appeal_chain, summ_chain],
        input_variables=["section", "desc"],
        output_variables=["summarized"],
    )

In [6]:
improve_summary_chain = improve_summary_chain()
improve_summary_chain({'section': summary, 'desc': description, 'format': '3 bullets and 1 sentence'})


{'section': {'section_name': 'resume summary',
  'result': "# Summary\n- Over 6 combined years of experience maintaining complex systems. Over 10 years of Python programming experience. Developed clean and maintainable code for the last 4 years.\n- Built projects involving Python, Rust, React, django/flask, and Angular for user interfaces (UI's), websites, automated trading systems, control system libraries, bioinformatic tools, and API wrappers.\n- Experience with git/GitHub, AWS, Docker, HTML/CSS, RUST API's, and extensive testing."},
 'desc': 'Vendelux (https://vendelux.com/) is the system of record for event marketing. Our software platform provides proprietary data that helps high-growth companies find the highest ROI events, conferences and trade shows to attend and sponsor. We have built a product that customers describe as an “event marketer’s dream” and are backed with venture capital funding and top-tier clients.\n\nWe are now hiring more engineers to help us build the best p

# Building a summary section from scratch

Using the improved job history section, artifacts should be *extrapolated*. These artifacts should include stories, facts or statements that make the summary appear unique, standout from typical candidates, and convey competence to a recruiter.

Multiple chains will be needed to extrapolate data in parallel and the result should be several lists of candidate clauses. The generated clauses will be iteratively reduced and fed to `improve_summary_chain()`.

## Candidate Clauses
- Extract "stories" from job experience
- "What's 3 things a recruiter should know about the candidate from this job experience?"

## Preparing Data

### Prepare skills

In [31]:
from util import job_requirement_chain
_req_chain = job_requirement_chain()
skills = _req_chain({'desc': description})

# TODO: simplify API for extracting skills
skills = skills['requirements'].skills
skills

['Python',
 'Django',
 'React',
 'Snowflake',
 'Software Engineering',
 'Agile Software Development',
 'Web Application Development',
 'Code Documentation',
 'Teamwork',
 'Fast Learner',
 'Results-driven',
 'English Proficiency']

### Separate job experience sections

In [22]:
experiences = chunk_markdown(sections['history']['result'])
experiences

[Document(page_content='Responsible for maintaining electrical and instrument systems for 11 F-18E aircraft totaling $715 million. Additionally, responsible for final check and troubleshooting of aircraft before flight.  \n- Assisted primary supervisor in management of maintenance programs leading to no hits on a major inspection.\n- Expertly managed tool program of over $175,000 by training coworkers on expectations and proper procedures leading to no hits across 2 major and 3 routine inspections.\n- Awarded for showing expertise and initiative for reinstalling an incorrectly installed part under stressful conditions.\n- Lead and mentored a group of 3 individuals in flight deck operations and proper flight deck cleaning procedures as supervisor leading to no accidents and rejuvenating a stagnant work center.', metadata={'Header 1': 'Experience', 'Header 2': 'Aviation Electrician, US Navy, Aug 2020 - Present (2 years 11 months)'}),
 Document(page_content="Passionate about leveraging te

## Extract "stories"

This chain extracts stories from a job experience section that exemplifies the given attribute.

In [97]:
from pydantic import BaseModel
from typing import List

class Stories(BaseModel):
    stories: List[str] = []

In [120]:
from langchain.output_parsers import PydanticOutputParser

def extract_stories_chain() -> LLMChain:
    parser = PydanticOutputParser(pydantic_object=Stories)
    _format_instructions = parser.get_format_instructions()
    
    prompt = PromptTemplate(
        template="""
        Extract {artifacts} from the following resume experience section that demonstrate {attribute}.
        
        Respond with 'None' for Python if there are no relevant {artifacts} to {attribute}.
        
        Here is the resume section (surrounded by ``):
        
        `{section}`
        
        {format_instructions}
        """,
        input_variables=["attribute", "section"],
        partial_variables={
            'format_instructions': _format_instructions,
            'artifacts': 'stories',
        })
    
    return LLMChain(llm=ChatOpenAI(temperature=.2, model_name="gpt-3.5-turbo"),
                    output_key='stories',
                    prompt=prompt,
                    output_parser=parser)

In [121]:
stories_chain = extract_stories_chain()

In [108]:
stories_chain({'section': experiences[0], 'desc': description, 'attribute': skills[0]})


{'section': Document(page_content='Responsible for maintaining electrical and instrument systems for 11 F-18E aircraft totaling $715 million. Additionally, responsible for final check and troubleshooting of aircraft before flight.  \n- Assisted primary supervisor in management of maintenance programs leading to no hits on a major inspection.\n- Expertly managed tool program of over $175,000 by training coworkers on expectations and proper procedures leading to no hits across 2 major and 3 routine inspections.\n- Awarded for showing expertise and initiative for reinstalling an incorrectly installed part under stressful conditions.\n- Lead and mentored a group of 3 individuals in flight deck operations and proper flight deck cleaning procedures as supervisor leading to no accidents and rejuvenating a stagnant work center.', metadata={'Header 1': 'Experience', 'Header 2': 'Aviation Electrician, US Navy, Aug 2020 - Present (2 years 11 months)'}),
 'desc': 'Vendelux (https://vendelux.com/) 

In [106]:
stories_chain({'section': experiences[0], 'artifacts': 'stories', 'desc': description, 'attribute': skills[8]})

{'section': Document(page_content='Responsible for maintaining electrical and instrument systems for 11 F-18E aircraft totaling $715 million. Additionally, responsible for final check and troubleshooting of aircraft before flight.  \n- Assisted primary supervisor in management of maintenance programs leading to no hits on a major inspection.\n- Expertly managed tool program of over $175,000 by training coworkers on expectations and proper procedures leading to no hits across 2 major and 3 routine inspections.\n- Awarded for showing expertise and initiative for reinstalling an incorrectly installed part under stressful conditions.\n- Lead and mentored a group of 3 individuals in flight deck operations and proper flight deck cleaning procedures as supervisor leading to no accidents and rejuvenating a stagnant work center.', metadata={'Header 1': 'Experience', 'Header 2': 'Aviation Electrician, US Navy, Aug 2020 - Present (2 years 11 months)'}),
 'artifacts': 'stories',
 'desc': 'Vendelux

## "What's 3 things a recruiter should know about the candidate from this job experience?" 

In [118]:
from langchain.output_parsers import CommaSeparatedListOutputParser

def extract_three_things_chain() -> LLMChain:
    """ Accepts job experience, returns list of three things recruiter should know about this job experience. """
    parser = CommaSeparatedListOutputParser()
    format_instructions = parser.get_format_instructions()
    
    prompt = PromptTemplate(template="""
    What are three things from the following job experience summary make this candidate stand out from the crowd to a job recruiter.
    
    Here is the job experience summary (surrounded by ``):
    `{section}`
    
    
    {format_instructions}
    """,
                            input_variables=["section"],
                            partial_variables={'format_instructions': format_instructions}
    )

    llm = ChatOpenAI(temperature=.4, model_name='gpt-3.5-turbo')
    return LLMChain(llm=llm, output_parser=parser, prompt=prompt, output_key='three_things')

In [119]:
three_things_chain = extract_three_things_chain()

In [115]:
three_things_chain({'section': experiences[0]})

InvalidRequestError: The model `gpt-4` does not exist or you do not have access to it. Learn more: https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4.

## Generate all candidate clauses

In [122]:
clauses = []
for experience in experiences:
    _three_things = three_things_chain({'section': experience})['three_things']
    clauses.extend(_three_things)
    
    for skill in skills:
        _stories = stories_chain({'section': experience, 'desc': description, 'attribute': skill})
        clauses.extend(_stories['stories'].stories)


Blindly recursing through all skills takes too long and is overkill. Top three skills should be matched. Create a `SequentialChain` that filters skills, then feeds to `stories_chain`

### Filter clauses

In [132]:
clauses = set(clauses)

In [183]:
def generate_summary_chain() -> SequentialChain:
    """ Chain that generates resume summary from candidate clauses """
    llm = ChatOpenAI(temperature=0.4, model_name="gpt-3.5-turbo")
    
    # Narrow down candidate clauses
    top_snippets_prompt = PromptTemplate.from_template("""
    We will be filtering a list of bullet points.
    Out of the bullets, select the top 10 that will make the candidate stand out for the given job description.
    
    The returned bullets should reflect the original content, and not be an excerpt from the job description.
    Also, Return the top snippets as bullet points.
    
    This is the job description (surrounded by ``):
    `{desc}`
    
    This is the list of candidate snippets (surrounded by ``):
    `{snippets}`
    """)
    top_snippets_chain = LLMChain(prompt=top_snippets_prompt, llm=llm, output_key='refined_snippets')
    
    summary_prompt = PromptTemplate.from_template("""
    Using the given snippets, generate a brief resume summary as bullet points.
    
    Here is a list of snippets (surrounded by ``):
    `{refined_snippets}`
    """)
    summary_chain = LLMChain(prompt=summary_prompt, llm=llm, output_key='summary_overview')
    
    refine_prompt = PromptTemplate.from_template("""
    Refine the given list of bullet points to 3-5 main points so that it stands out to someone reading the given job description.
    
    Here is a list of bullet points (surrounded by ``):
    `{summary_overview}`
    
    Here is the job description (surrounded by ``):
    `{desc}`
    """)
    refine_chain = LLMChain(prompt=refine_prompt, llm=llm, output_key='refined_overview')
    
    return SequentialChain(chains=[top_snippets_chain, summary_chain, refine_chain],
                           input_variables=['desc', 'snippets'],
                           output_variables=['refined_overview'])

In [184]:
_generate_summary_chain = generate_summary_chain()

In [185]:
_generate_summary_chain({'desc': description, 'snippets': clauses,})

{'desc': 'Vendelux (https://vendelux.com/) is the system of record for event marketing. Our software platform provides proprietary data that helps high-growth companies find the highest ROI events, conferences and trade shows to attend and sponsor. We have built a product that customers describe as an “event marketer’s dream” and are backed with venture capital funding and top-tier clients.\n\nWe are now hiring more engineers to help us build the best platform for Event Marketers. We are looking for a Django expert to build and maintain our core platform.\n\nThe Django Engineer’s role is a full-time, remote position. This role reports to our Head of Product & Engineering.\n\nResponsibilities\n\n- Be an important part of our multinational, agile software development team\n- Implement high-quality solutions across multiple components of our platform\n- Write well-structured and documented code\n- Continuously update and expand our products\n\n\nQualifications\n\n- 5+ years of experience 